# 규제 선형 모델 

__< Good Regression Model ? >__<br><br>
이전 까지는 좋은 회귀 모델을 위해 <span style="color:red">RSS (실제 값 - 예측 값) 를 최소화 방법</span>만 고려했다.<br>
* 편향-분산 Trade off 를 유지 하며, 오류 값은 최소가 되어야함. 
                            ∥ 
* 지나치지 않고 적절하게 데이터에 적합해야 함.<br><br>

그러다 보니, 학습 데이터에 지나치게 맞추게 되고 회귀 계수가 쉽게 커지는 상황이 발생했다. <br> 
그러면, 예측 성능이 저하 되기 때문에  <span style="color:red">회귀 계수의 크기 제어</span>도 고려 해야 한다. <br><br>
즉, 회귀 계수의 크기를 제어해 과적합을 개선하려면 cost function 의 목표를 아래와 같이 설정 할 수 있다.
> 비용 함수 목표 = Min ( RSS ( W ) + alpha * || W || ²₂)

__목표 수식 분석__

* 비용함수 의 목표는 위 식을 최소화하는 W vector 를 찾는 것<br>W 파라미터 =  ω₀, ω₁ 등 회귀 계수<br><br>
* alpha 는 학습 데이터 적합 정도와 회귀 계수 값의 크기 제어를 수행하는 튜닝 파라미터 <br>
    * alpha = 0 인 경우 : W 가 커도  alpha * || W || ²₂= 0 이기 때문에 비용함수는  Min ( RSS ( W ) ) <br>
    * alpha = ∞ 인 경우 : alpha * || W || ²₂가 ∞ 가 되므로 비용 함수는 W를 0 에 가깝게 최소화 해야 함.<br>
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 그래야 RSS(W) 값이 작아지고 어느 정도 상쇄가 가능<br>
    
즉, alpha 를 0 에서 부터 지속적으로 값을 증가시키면 회귀 계수 값의 크기를 감소 시킬 수 있다. ( p.321 )<br><br>

__< About Regulation >__<br><br>
이처럼 비용 함수에 alpha 값으로 페널티를 부여해 회귀 계수 값의 크기를 감소시켜 과적합을 개선하는 방식을 __규제 Regulation__ 이라 부른다.
* L2 규제 : alpha * || W || ²₂와 같이 W 의 제곱에 대해 페널티를 부여하는 방식 ≒ Ridge Regression
* L1 규제 : alpha * || W || ₁  와 같이 W 의 절대 값에 대해 페널티를 부여하는 방식 ≒ Lasso Regression



## Ridge Regression

__< What is Ridge >__

alpha 에 L2 규제를 적용하는 릿지 회귀 구현 

__< 예제 >__
* 내장 dataset : boston 주택 가격 데이터 
* 성능 평가 : MSE , RMSE 

In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

boston = load_boston()

bostonDF = pd.DataFrame(boston.data,columns= boston.feature_names)
bostonDF['PRICE'] = boston.target

y_label = bostonDF['PRICE']
X_feature = bostonDF.drop(['PRICE'],axis=1,inplace=False)

# alpha = 10으로 설정해 릿지 회귀 수행
ridge = Ridge(alpha = 10)

# 성능평가 지표로 MSE 와 RMSE 사용
neg_mse_score = cross_val_score(ridge,X_feature,y_label,scoring="neg_mean_squared_error",cv=5)
rmse_score = np.sqrt(neg_mse_score * -1)

print('5 folds 의 개별 Negative MSE score: ',np.round(neg_mse_score,3))
print('5 folds 의 개별 RMSE score: ',np.round(rmse_score,3))
print('5 folds 의 평균 RMSE: ',np.round(np.mean(rmse_score),3))


5 folds 의 개별 Negative MSE score:  [-11.422 -24.294 -28.144 -74.599 -28.517]
5 folds 의 개별 RMSE score:  [3.38  4.929 5.305 8.637 5.34 ]
5 folds 의 평균 RMSE:  5.518


회귀 성능 평가 지표에서 수치가 낮을 수록 좋은 모델이라 했었다 .<br>
즉, 앞선 파일의 예제와 비교해 보아, 규제가 없는 LinearRegression 보다 Ridge 가 훨씬 좋은 성능을 보여준다.  <br><br>

이제는 __Ridge 의 alpha 값을 변화 시키면서 RMSE 값의 변화__를 살펴보자.


In [5]:
alphas = [0,0.1,1,10,100]

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    
    neg_mse_scores = cross_val_score(ridge,X_feature,y_label,scoring='neg_mean_squared_error',cv=5)
    avg_rmse_score = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpha {0} 일 때 5 folds 의 평균 RMSE 값은 {1:.3f}'.format(alpha,avg_rmse_score),'\n')

alpha 0 일 때 5 folds 의 평균 RMSE 값은 5.829 

alpha 0.1 일 때 5 folds 의 평균 RMSE 값은 5.788 

alpha 1 일 때 5 folds 의 평균 RMSE 값은 5.653 

alpha 10 일 때 5 folds 의 평균 RMSE 값은 5.518 

alpha 100 일 때 5 folds 의 평균 RMSE 값은 5.330 



이번에는 __Ridge 의 alpha 값 변화에 따른 회귀 계수 값의 변화__ 를 살펴보자.<br><br>
Plotly 공식문서 ( Graph Objects 사용 )
- [Subplots](https://plotly.com/python/subplots/) 
- [Horizontal bar](https://plotly.com/python/horizontal-bar-charts/#horizontal-bar-chart-with-gobar) 
- [Axes ](https://plotly.com/python/axes/)

를 활용해서 시각화 하였다.

In [72]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1,cols=5,
                        subplot_titles=("alpha 0", "alpha 0.1", "alpha 1", "alpha 10","alpha 100"))

coeff_df = pd.DataFrame()

for pos,alpha in enumerate(alphas):
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_feature,y_label)
    
    # fit 시킨 뒤 회귀 계수로 Series 를 만든 뒤 DataFrame 에 각 alpha 따라 붙이기 
    # Dataframe 의 index 는 boston dataset 의 feature 들 
    coeff = pd.Series(data=ridge.coef_,index=X_feature.columns)
    colname = 'alpha:'+str(alpha)
    coeff_df[colname]=coeff
    
    
    # 막대 그래프로 각 alpha 값에서의 피처별 회귀 계수를 시각화 , 높은 순으로 
    coeff = coeff.sort_values(ascending=True)
    
    fig.add_trace(
        go.Bar(x=coeff.values,y=coeff.index, orientation='h'),
        row=1, col=pos+1
    )
    
    fig.update_xaxes(range=[-2,2])

fig.show()

해당 그래프를 보면, alpha 값이 계속 증가 할 수록 회귀 계수 값은 지속적으로 작아짐을 알 수 있다.

In [62]:
coeff_df

,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
CRIM,-0.108011,-0.107474,-0.104595,-0.101435,-0.102202
ZN,0.046420,0.046572,0.047443,0.049579,0.054496
INDUS,0.020559,0.015999,-0.008805,-0.042962,-0.052826
CHAS,2.686734,2.670019,2.552393,1.952021,0.638335
NOX,-17.766611,-16.684645,-10.777015,-2.371619,-0.262847
RM,3.809865,3.818233,3.854000,3.702272,2.334536
AGE,0.000692,-0.000269,-0.005415,-0.010707,0.001212
DIS,-1.475567,-1.459626,-1.372654,-1.248808,-1.153390
RAD,0.306049,0.303515,0.290142,0.279596,0.315358
TAX,-0.012335,-0.012421,-0.012912,-0.013993,-0.015856


## Lasso Regression


__< What is Lasso >__

W 의 절댓값에 페널티를 부여하는 L1 규제를 적용한 선형 회귀 <br><br>

L2 규제가 회귀 계수의 크기를 감소 시키는데 반해, L1 규제는 <span style="color:red">불필요한 회귀 계수를 급격하게 감소</span>시켜 0 으로 만들고 제거한다.<br>
그래서 L1 규제는 적절한 피처만 회귀에 포함시키는 피처 선택의 특성을 가진다. <br><br>
__< 예제 >__<br>
* 이번에도 alpha 값을 변화 시키면서 RMSE 값과 각 피처의 회귀 계수에 대해 알아보자. <br>
  이를 수행하기 위해 별도의 함수 get_linear_reg_eval() 를 만들었으며, 필요한 생성 파라미터는 변수명으로 명시해 두었다.
  
  
* 릿지 회귀에서 사용했던 datset 을 그대로 사용

In [88]:
from sklearn.linear_model import Lasso,ElasticNet

def get_linear_reg_eval(model_name, params=None, X_feature=None, y_label=None, verbose=True , return_coeff=True):
    
    dataframe = pd.DataFrame()
    
    for param in params:
        
        if model_name == 'Ridge': model = Ridge(alpha=param)
        if model_name == 'Lasso': model = Lasso(alpha=param)
        if model_name == 'ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
            
        # RMSE 예측 성능 
        neg_mse_scores = cross_val_score(model,X_feature,y_label,scoring='neg_mean_squared_error',cv=5)
        avg_rmse_score = np.mean(np.sqrt(neg_mse_scores *  -1))
        
        print('alpha {0} 일 때. 5 folds 의 평균 RMSE 값은 {1:.3f}'.format(param,avg_rmse_score),'\n')
        
        # 회귀 계수 추출 
        model.fit(X_feature,y_label)
        
        if return_coeff:
            coeff = pd.Series(data=model.coef_,index=X_feature.columns)
            colname = 'alpha:'+str(param)
            dataframe[colname] = coeff
            coeff = coeff.sort_values(ascending=True)

    return dataframe

In [97]:
# lasso 모델 사용 

lasso_alpha = [0.01,0.05,0.1,0.3,0.5,1,5]

get_linear_reg_eval('Lasso',params=lasso_alpha,X_feature=X_feature,y_label=y_label)



alpha 0.01 일 때. 5 folds 의 평균 RMSE 값은 5.740 

alpha 0.05 일 때. 5 folds 의 평균 RMSE 값은 5.628 

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 5.615 

alpha 0.3 일 때. 5 folds 의 평균 RMSE 값은 5.628 

alpha 0.5 일 때. 5 folds 의 평균 RMSE 값은 5.669 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 5.776 

alpha 5 일 때. 5 folds 의 평균 RMSE 값은 6.375 



,alpha:0.01,alpha:0.05,alpha:0.1,alpha:0.3,alpha:0.5,alpha:1,alpha:5
CRIM,-0.106228,-0.098935,-0.097894,-0.091461,-0.083289,-0.063437,-0.000000
ZN,0.046860,0.048826,0.049211,0.049744,0.049544,0.049165,0.038467
INDUS,0.006474,-0.041355,-0.036619,-0.017829,-0.005253,-0.000000,-0.000000
CHAS,2.504190,1.761631,0.955190,0.000000,0.000000,0.000000,0.000000
NOX,-14.394478,-1.044929,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
RM,3.814186,3.836618,3.703202,3.095828,2.498212,0.949811,0.000000
AGE,-0.001818,-0.011858,-0.010037,-0.000963,0.003604,0.020910,0.031679
DIS,-1.422155,-1.202690,-1.160538,-1.040480,-0.936605,-0.668790,-0.000000
RAD,0.298526,0.271034,0.274707,0.280509,0.277451,0.264206,0.000000
TAX,-0.012627,-0.014028,-0.014570,-0.015277,-0.015442,-0.015212,-0.007669


RSME 는 낮을 수록 높은 예측 성능을 가짐으로, 0.1 에서 가장 좋은 성능을 보인다. <br><br>
아래 dataframe 을 보면, <br>
* NOX 피처 :  alpha 0.01 → 0.05 회귀 계수가 급격하게 감소하다가 0 에 도달
* CHAS 피처 : alpha 0.1 → 0.3 회귀 계수의 감소로 0 에 도달
* INDUS 피처 : alpha 0.5 → 1 회귀 계수의 감소로 0 에 도달 

즉, 회귀 계수가 0인 피처는 회귀 식에서 제외되면서 피처 선택의 효과를 얻을 수 있는 것이 __Lasso Regression__ 이다.

## Elastic Net Regression 

__< What is Elastic Net >__

엘라스틱넷은 L2 규제와 L1 규제를 결합한 회귀이다. 따라서, 엘라스틱넷의 회귀 비용 함수의 목표는
> Min ( RSS ( W ) + alpha 2 * || W || ²₂+ alpha 1 * || W || ₁) 을 만족하는 W 를 찾는 것 

먼저, __라쏘 회귀__ 에 의해 서로 상관관계가 높은 피처들의 경우, 중요 피처만을 설렉션하고 다른 피처들은 모두 회귀 계수를 0으로 만드려는 성향이 강하다. 그래서 alpha 값에 따라 회귀 계수의 값이 급격히 변동 될 수 있는데 이를 완화 하기 위해 __L2 규제를 라쏘 회귀에 추가__ 한 것.<br><br>
두 규제가 결합되었기 때문에 상대적으로 수행 시간은 오래걸린다.
<br><br>

__< How to use in sklearn >__

앞서 말한대로, ElasticNet = a * L1 + b * L2 로 정의 될 수 있기 때문에 

생성 파라미터 
* alpha = <span style="color:red">a + b </span>( a 는 L1 규제의 alpha 값, b 는 L2 규제의 alpha 값 )<br><br>
* l1_ratio : a / (a+b) = L1 규제의 비율 ⇒ a 가 0 이면 L2 규제와 동일 , b 가 0 이면 L1 규제와 동일 
<br><br>

__< 예제 >__

앞서 연습한 릿지와 라쏘 회귀 처럼 alpha 값을 변화 시키면서 RMSE 값과 각 피처의 회귀 계수를 출력해보자.<br>
미리 만들어 두었던 get_linear_reg_eval() 함수를 이용한다.

In [100]:
elastic_alpha = [0.07,0.1,0.5,1,3]
get_linear_reg_eval('ElasticNet',params=elastic_alpha,X_feature=X_feature,y_label=y_label)

alpha 0.07 일 때. 5 folds 의 평균 RMSE 값은 5.542 

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 5.526 

alpha 0.5 일 때. 5 folds 의 평균 RMSE 값은 5.467 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 5.597 

alpha 3 일 때. 5 folds 의 평균 RMSE 값은 6.068 



,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
CRIM,-0.099468,-0.099213,-0.089070,-0.073577,-0.019058
ZN,0.050107,0.050617,0.052878,0.052136,0.038268
INDUS,-0.044855,-0.042719,-0.023252,-0.000000,-0.000000
CHAS,1.330724,0.979706,0.000000,0.000000,0.000000
NOX,-0.175072,-0.000000,-0.000000,-0.000000,-0.000000
RM,3.574162,3.414154,1.918419,0.938789,0.000000
AGE,-0.010116,-0.008276,0.007760,0.020348,0.043446
DIS,-1.189438,-1.173647,-0.975902,-0.725174,-0.031208
RAD,0.278880,0.283443,0.300761,0.289299,0.146846
TAX,-0.014522,-0.014814,-0.016046,-0.016218,-0.011417


alpha 0.5 에서 가장 좋은 예측 성능을 보이고 있다.<br><br>
그 아래 dataframe 을 살펴보면, 회귀 계수가 0 이 되는 피처가 아직 존재하긴 하나 Lasso 보다는 적음을 알 수 있다.

## 선형 회귀 모델을 위한 데이터 변환 

선형 회귀 모델은 피처값과 타깃값의 분포가 정규 분포 형태를 띄는 것이 좋다. <br>
특히 타깃값의 경우 정규 분포 형태가 아니라, 분포가 치우진 왜곡된 형태일 경우 예측 성능에 부정적인 영향을 미칠 가능성이 매우 높다.<br><br>
그래서 선형 회귀 모델 적용 전에 <u>스케일링 / 정규화 작업</u> 은 필수적이다. <br>
여기서 문제는 피처값과 타깃값에 위 작업을 수행하는 방법에 약간의 차이가 있다. <br><br>
__< 피처 dataset 에 적용하는 방법 >__<br>

* StandardScaler 로 평균 0 , 분산 1 인 표준 정규 분포로 변환 or MinMaxScaler 로 최솟값 0 , 최댓값 1 인 값으로 정규화 
* 위 방법을 통해 성능 향상이 없을 경우, 수행한 dataset 에 다항 변환 적용
* Log transformation : 가장 유용한 방법 / 원래 값에 log 함수를 적용하면 정규 분포에 가까운 형태로 변환됨 
    * Scaler 는 성능 향상을 기대하기 어려움 
    * Polynomial 은 피처가 많을 경우 다항 변환 시 피처의 갯수가 기하급수로 늘어나서 과적합 문제가 우려됨 

__< 타켓 dataset 에 적용하는 방법 >__<br>
* 항상 Log transformation 적용 
<br><br>

이제, boston dataset 을 가지고 명시한 변환 방법 = Standard, MinMax, Log 변환을 차례로 적용한 후에<br>
Ridge 회귀로 각 변환 별 alpha 값 들을 조정하여 RMSE 의 예측 성능을 비교해 보자.<br><br>
이를 위해 get_scaled_data() 함수를 생성한다.
<br><br>

__< get_scaled_data() 사용자 정의 함수 >__



생성 파라미터
* method : 변환 방법 결정 
* p_degree : 다항식 차수 결정 
* input_data : 변환 하고자 하는 데이터 


In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
from sklearn.preprocessing import PolynomialFeatures

def get_scaled_data(method='None',p_degree='None', input_data='None'):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == "MinMax":
        scaled_data = MinMaxScaler().fit_transform(input_data)
    else:
        # 원래 log 변환은 np.log() 를 사용하지만, 언더 플로우 발생을 방지하기 위해서 1 + log() 인 log1p() 사용 
        scaled_data = np.log1p(input_data)
        
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree,include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

이제 사전에 만들어 두었던 
* get_linear_reg_eval() = 원하는 규제 선형 모델을 선택하여 여러 alpha 값에 따라 변화하는 RMSE 와 회귀 계수를 파악하는 함수
* get_scaled_data() = 데이터 변환 방식과 다항식 차수를 임의로 선택하여 그에 맞게 주어진 데이터를 전처리 하는 함수
<br>

를 이용 해보자.

참고로, 데이터 변환은 (None,None) &nbsp; (Standard,None)  &nbsp; (Standard,2)&nbsp; (MinMax,None)  &nbsp; (MinMax,2) &nbsp; (Log,None) &nbsp;  총 6가지를 사용한다.

In [107]:
# Ridge 의 alpha 값을 다르게 적용하고 데이터 변환 방법을 선택하여 RMSE 추출 

alphas = [0.1,1,10,100]

scale_methods = [(None,None),('Standard',None),('Standard',2) , ('MinMax',None) ,('MinMax',2) , ('Log',None) ]

for scale_method in scale_methods:
    X_feature_scaled = get_scaled_data(method=scale_method[0],p_degree=scale_method[1],input_data=X_feature)
    print('\n### 변환 유형: {0} / Polynomial degree: {1} ###\n'.format(scale_method[0],scale_method[1]))
    print('=======================================================')
    
    # Ridge 회귀를 사용해서 alpha 값에 변화를 주었고, scaled 된 dataset 을 넘겨주었고, 회귀 계수는 사용안하기 때문에 False
    get_linear_reg_eval('Ridge',params=alphas,X_feature=X_feature_scaled,y_label=y_label, verbose=False, return_coeff=False)


### 변환 유형: None / Polynomial degree: None ###

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 4.770 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 4.676 

alpha 10 일 때. 5 folds 의 평균 RMSE 값은 4.836 

alpha 100 일 때. 5 folds 의 평균 RMSE 값은 6.241 


### 변환 유형: Standard / Polynomial degree: None ###

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 5.826 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 5.803 

alpha 10 일 때. 5 folds 의 평균 RMSE 값은 5.637 

alpha 100 일 때. 5 folds 의 평균 RMSE 값은 5.421 


### 변환 유형: Standard / Polynomial degree: 2 ###

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 8.827 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 6.871 

alpha 10 일 때. 5 folds 의 평균 RMSE 값은 5.485 

alpha 100 일 때. 5 folds 의 평균 RMSE 값은 4.634 


### 변환 유형: MinMax / Polynomial degree: None ###

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 5.764 

alpha 1 일 때. 5 folds 의 평균 RMSE 값은 5.465 

alpha 10 일 때. 5 folds 의 평균 RMSE 값은 5.754 

alpha 100 일 때. 5 folds 의 평균 RMSE 값은 7.635 


### 변환 유형: MinMax / Polynomial degree: 2 ###

alpha 0.1 일 때. 5 folds 의 평균 RMSE 값은 5.298 

alpha 1 일 때. 5 fol

해당 결과에서 RMSE 가 4.xx 인 경우를 살펴보자. 
* MinMax  /  Polynomial 2  /  alpha = 1<br><br>
* Standard / Polynomial 2 /  alpha = 100 <br><br>
* Log / Polynomial None / alpha = 0.1 , 1 , 10 <br><br>

즉, 단순 표준 정규 분포 or 최대 / 최소값 정규화 만으로는 RMSE 예측 성능을 향상 시킬 수 없었다. <br>
물론 2차 다항식에서는 alpha 값에 따라 예측 성능이 우수한 경우도 있었으나, 피처 갯수가 많을 경우 다항식 변환을 적용하기 어렵기 때문에 한계가 있다.<br><br>
반면, Log 변환은 alpha 값에 상관없이 대부분 좋은 성능을 보이기 때문에 
> 선형 회귀를 적용하려는 dataset 에 data 의 분포가 심하게 왜곡 되어 있을 경우, __로그 변환__을 적용하는 것이 좋다 !!! 
